In [20]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

import os
import numpy as np
import pandas as pd
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow .keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
smooth = 1e-15
H = 256
W = 256

In [4]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding = 'same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, 3, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2,2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    # print(inputs.shape)
    x = Conv2DTranspose(num_filters, 2, strides = 2, padding = 'same')(inputs)
    # print(x.shape)
    x = Concatenate()([x, skip_features])
    # print(x.shape)
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # print(s1.shape, s2.shape, s3.shape, s4.shape)
    # print(p1.shape, p2.shape, p3.shape, p4.shape)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    # print(d4.shape)

    outputs = Conv2D(1, 1, padding = 'same', activation ='sigmoid')(d4)
    # print(outputs.shape)

    model = Model (inputs, outputs, name = "UNET")
    return model

  
input_shape = (256, 256, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [5]:
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection =  tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path, split = 0.2):
    images = sorted(glob(os.path.join(path, "images","*.png")))
    masks = sorted(glob(os.path.join(path, "masks","*.png")))
    split_size = int(len(images)*split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)
    return (train_x,train_y), (valid_x,valid_y), (test_x,test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch = 2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

In [6]:
np.random.seed(42)
tf.random.set_seed(42)

create_dir("files")

batch_size = 16
lr = 1e-4
num_epochs = 500

model_path = os.path.join("files", "model.keras")
csv_path = os.path.join("files", "log.csv")
dataset_path = "/kaggle/input/brain-tumor-segmentation"

(train_x,train_y), (valid_x,valid_y), (test_x,test_y) = load_dataset(dataset_path)
print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")
print(f"Test: {len(test_x)} - {len(test_y)}")

train_dataset = tf_dataset(train_x, train_y, batch = batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch = batch_size)

Train: 1840 - 1840
Valid: 612 - 612
Test: 612 - 612


In [7]:
model = build_unet((H, W, 3))
model.compile(loss = dice_loss, optimizer=Adam(lr), metrics=[dice_coef])

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs = num_epochs,
    validation_data = valid_dataset,
    callbacks = callbacks
)

Epoch 1/500


2024-03-15 16:44:05.025753: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,64,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,64,256,256]{3,2,1,0}, f32[16,64,256,256]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-03-15 16:44:06.042232: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.016701901s
Trying algorithm eng0{} for conv (f32[64,64,3,3]{3,2,1,0}, u8[0]{0}) custom-call(f32[16,64,256,256]{3,2,1,0}, f32[16,64,256,256]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-03-15 16:44:08.735524: E external/

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 809ms/step - dice_coef: 0.0786 - loss: 0.9214
Epoch 1: val_loss improved from inf to 0.98238, saving model to files/model.keras
115/115 ━━━━━━━━━━━━━━━━━━━━ 300s 1s/step - dice_coef: 0.0789 - loss: 0.9211 - val_dice_coef: 0.0176 - val_loss: 0.9824 - learning_rate: 1.0000e-04
Epoch 2/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 811ms/step - dice_coef: 0.1868 - loss: 0.8132
Epoch 2: val_loss did not improve from 0.98238
115/115 ━━━━━━━━━━━━━━━━━━━━ 105s 912ms/step - dice_coef: 0.1869 - loss: 0.8131 - val_dice_coef: 0.0165 - val_loss: 0.9835 - learning_rate: 1.0000e-04
Epoch 3/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 811ms/step - dice_coef: 0.2483 - loss: 0.7517
Epoch 3: val_loss improved from 0.98238 to 0.93073, saving model to files/model.keras
115/115 ━━━━━━━━━━━━━━━━━━━━ 107s 926ms/step - dice_coef: 0.2484 - loss: 0.7516 - val_dice_coef: 0.0693 - val_loss: 0.9307 - learning_rate: 1.0000e-04
Epoch 4/500
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 812ms/step - dice_coef: 0.3116 - 

In [8]:
from tqdm import tqdm
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score

In [13]:
def save_results(image, mask, y_pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = np.concatenate([mask, mask, mask], axis=-1)
    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1)
    
    y_pred = y_pred * 255
    
    line = np.ones((H,7,3)) * 255
    
    cat_images = np.concatenate([image, mask, y_pred], axis=1)
    cv2.imwrite(save_image_path, cat_images)

In [22]:
np.random.seed(42)
tf.random.set_seed(42)

create_dir("results")

model = tf.keras.models.load_model("/kaggle/working/files/model.keras", custom_objects = {"dice_loss": dice_loss, "dice_coef": dice_coef})

Score = []
for x, y in tqdm(zip(test_x, test_y), total = len(test_y)):
    name = x.split("/")[-1]
    
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (W, H))
    x = image/255.0
    x = np.expand_dims(x,axis = 0)
    
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, (W,H))
    
    y_pred = model.predict(x, verbose=0)[0]
    y_pred = np.squeeze(y_pred, axis = -1)
    y_pred = y_pred >= 0.5
    y_pred = y_pred.astype(np.int32)
    
    save_image_path = os.path.join("/kaggle/working/results",name)
    save_results(image, mask, y_pred, save_image_path)
    
    mask = mask/255.0
    mask = (mask>0.5).astype(np.int32).flatten()
    y_pred = y_pred.flatten()
    
    f1_value = f1_score(mask, y_pred, labels=[0,1], average = 'binary')
    jac_value = jaccard_score(mask, y_pred, labels=[0, 1], average="binary")
    recall_value = recall_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    precision_value = precision_score(mask, y_pred, labels=[0, 1], average="binary", zero_division=0)
    Score.append([name, f1_value, jac_value, recall_value, precision_value])
    
score = [s[1:]for s in Score]
score = np.mean(score, axis=0)
print(f"F1: {score[0]:0.5f}")
print(f"Jaccard: {score[1]:0.5f}")
print(f"Recall: {score[2]:0.5f}")
print(f"Precision: {score[3]:0.5f}")

df = pd.DataFrame(Score, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
df.to_csv("files/score.csv")

100%|██████████| 612/612 [02:11<00:00,  4.65it/s]

F1: 0.76135
Jaccard: 0.67523
Recall: 0.75806
Precision: 0.82129
